In [1]:
import requests
import uuid
import xlrd
import yaml
from xlutils.copy import copy
from tqdm import tqdm

In [2]:
def read_yaml(yaml_path):
    with open(yaml_path, encoding="utf-8", mode="r") as f:
        result = yaml.load(stream=f,Loader=yaml.FullLoader)
        return result

In [3]:
def excel(result):

    wb = xlrd.open_workbook(result['example_excel_path'])
    st = wb.sheet_by_index(0)  # sheet索引从0开始

    excel = copy(wb)

    write_excel = excel.get_sheet(0)

    return excel, write_excel, st

In [4]:
def main(line, write_excel, row, msg):
    pop = line[:8] + "-" + line[8:12] + "-" + line[12:16] + "-" + line[16:20] + "-" + line[20:]
    url = "https://bdcs-file-2.ykt.cbern.com.cn/zxx/ndrs/resources/{}.json".format(pop)

    pop = requests.get(url).json()

    try:
        id = pop['relations']['lesson_plan_design'][0]['id']
    except:
        return 0
    
    school = pop['custom_properties']['school_name']

    teacher = pop['author']

    msgs = msg.split("-")
    if len(msgs) == 5:
        write_excel.write(row, 1, msgs[0]) #学段

        write_excel.write(row, 2, msgs[1]) #学科

        write_excel.write(row, 3, msgs[2]) #教材

        write_excel.write(row, 4, msgs[3]+msgs[4].strip()) #年级
    else:
        return 0
        # write_excel.write(row, 1, msgs[0])

        # write_excel.write(row, 2, msgs[1])

        # write_excel.write(row, 3, msgs[2])

        # write_excel.write(row, 4, msgs[3])
        
    
    write_excel.write(row, 0, '国家中小学智慧教育平台')

    write_excel.write(row, 9, school)

    write_excel.write(row, 10, teacher)

    for e in pop['relations']['lesson_plan_design'][0]['ti_items']:
        if e['ti_storages'][0].find(".pdf") > 0:
            dlink = e['ti_storages'][0]
            break
        else:
            continue

    write_excel.write(row, 11, dlink)

    write_excel.write(row, 12, str(uuid.uuid4()))
    return 1

In [5]:
if __name__ == '__main__':

    result = read_yaml("../../settings/settings.yaml")
    new_excel, write_excel, st = excel(result)

    with open(result['resource_path'], "r",encoding="utf-8") as f:
        lines = f.readlines()
    # lines = ['课文_语文园地四_书写提示+日积月累_8aee80fe6a609d81016a6459b551190a_小学-语文-统编版-一年级-下册','课文_语文园地四_书写提示+日积月累_8aee80076a5650cf016a591dc2d9145e_小学-语文-统编版-一年级-下册']

    for l in tqdm(lines):
        splits = l.split("_")

        m = main(splits[-2].strip(), write_excel, st.nrows, splits[-1])
        if m == 0 :
            continue
        else:
            for e in range(len(splits) - 2):
                write_excel.write(st.nrows,5 + e,splits[e])
            
            st.nrows = st.nrows + 1
            
    new_excel.save(result['teaching_plan_excel_path'])

100%|██████████| 6271/6271 [37:09<00:00,  2.81it/s]
